In [142]:
import json
import re
import typing

class MapperLocation:
    def __init__(self, location, column_name, coalesce_value=None, if_missing='ignore', attempt_json_serializing=True, delimiter='::',  convert_function=None):


        
        """
        self.location = location
        self.column_name = column_name
        self.coalesce_value=coalesce_value
        self.if_missing=if_missing
        self.attempt_json_serializing=True
        self.delimiter=delimiter
        self.convert_function=convert_function
        
class RequiredElementMissingError(AttributeError):
    pass

class DictMapper:
    def __init__(self, in_dict_list: list, mapper: dict) -> None:
        self.in_dict_list = in_dict_list
        self.mapper = mapper
        self.table_names = [key for key in mapper]

        self.re_check_list_base_location = r"^.+(?=\[\])"
        self.re_check_list_specific_element = r"[a-zA-Z]+\[[0-9]+\]"
        self.re_check_list_unlimited_element = r"[a-zA-Z]+(?=\[\])"
        self.re_list_location_name = r"[a-zA-Z]+(?=\[)"
        self.re_list_location_index = r"(?<=\[)[0-9]+(?=\])"


    def get_records(self, table_name: str=None, extra_data=None) -> typing.Generator:
        if extra_data is None:
            extra_data = {}

        if table_name in self.table_names:
            # yield self._dump_to_records(in_rows=self.in_dict_list, mapper_set=self.mapper.get(table_name), extra_data=extra_data)
            return self._dump_to_records(in_rows=self.in_dict_list, mapper_set=self.mapper.get(table_name), extra_data=extra_data)
        else:
            raise ValueError(f'{table_name} is not defined in mapper. We got {self.table_names}')

    def auto_flatten_records(self, prefix: str=str(), stringify_list=False, force_lowercase=False) -> typing.Generator:
        for single_dict in self.in_dict_list:
            row_needs_expanding = True
            consider_list = [single_dict]

            while row_needs_expanding:
                if len(consider_list) == 0:
                    row_needs_expanding = False

                for element in consider_list:
                    check_list = self._expand_dict(element)
                    if isinstance(check_list, list) and len(check_list) > 1:
                        consider_list.extend(check_list)
                    else:
                        # flat_list.extend(check_list)
                        print(type(check_list))
                        if check_list is not None:
                            yield check_list
                    consider_list.remove(element)
        # for in_dict in self.in_dict_list:
        #     yield self._flatten_single_dict(root_dict=in_dict, prefix=prefix, stringify_list=stringify_list, force_lowercase=force_lowercase)

    def _flatten_single_dict(self, root_dict: dict, prefix: str=str(), stringify_list=False, force_lowercase=False) -> dict:
        """Takes a nested dictionary and "flattens" it into a single level key:value set. Object children in elements will have a prefix

        Eg: flatten_dict({"a": {"b": {"c": 1}}) == {"a_b_c": 1}
        
        root_dict: input dict

        prefix: str = name to append to returned elements    

        stringify_list: bool = False = If we encounter a list object, do we stringify it or leave it be?

        force_lowercase: bool = False = Force keys to lowercase (including prefix)

        Returns:
        dict = flattened object
        """

        root_dict = root_dict.copy()
        flattened = {}

        for k, v in root_dict.items():
            new_key = f"{prefix}{k}"
            new_key = new_key.lower() if force_lowercase else new_key
            
            # Take in any types that are not nested dictionaries.
            if type(v) not in [dict, list]:
                flattened.update({new_key: v})
            elif type(v) == dict:
                flattened.update(self._flatten_single_dict(root_dict=v, prefix=new_key + '_', force_lowercase=force_lowercase))
            # Only list should exist here.
            elif type(v) == list:
                # warnings.warn(f'{new_key} is a list')
                if stringify_list == True:
                    flattened.update({new_key: json.dumps(v)})
                else:
                    flattened.update({new_key: v})

        return flattened
    
    def _expand_dict(self, in_dict):
        flat_list = []
        row_was_expanded = False
        for k, v in in_dict.items():
            if type(v) == list:
                for row in v:
                    subroot = in_dict.copy()
                    subdata = self._flatten_single_dict(row, prefix=f'{k}_')
                    subroot.update(subdata)
                    # in_dict = subroot.copy()
                    del subroot[k]
                    flat_list.append(subroot)
                    row_was_expanded = True

        if not row_was_expanded:
            flat_list.append(in_dict)

    def _dump_single_row(self, in_row: list, mapper_set: list, extra_data: dict=None, extra_data_in_front:bool=False, ignore_location_str: str=None):
        extra_data = {} if extra_data is None else extra_data
        
        output_row = {}

        if extra_data_in_front:
            output_row.update(extra_data)
        
        first_column = next(iter(mapper_set)).column_name

        # for column, location in column_set.items():
        for mapperlocation in mapper_set:
            column = mapperlocation.column_name
            location = mapperlocation.location
            delimiter = mapperlocation.delimiter
            convert_function = mapperlocation.convert_function
            coalesce_value = mapperlocation.coalesce_value

            if convert_function is not None:
                coalesce_value = convert_function(coalesce_value)

            # Hack to remove unneeded location string as we now have no reference to it.
            if ignore_location_str is not None:
                # ignore_location_str = ignore_location_str #+ f'[]{mapperlocation.delimiter}'
                location = location.replace(ignore_location_str, '')

            # If we only need to pull a single element.
            try:
                location_elements = location.split(delimiter)

                if location_elements[0] == 'root' and len(location_elements) == 1:
                    if convert_function is not None:
                        in_row = convert_function(in_row)

                    output_row.update({column: in_row})

                current_object = in_row
                
                for single_location in location_elements:
                    # Ignore the root element
                    if single_location == 'root':
                        continue

                    # If we specify we want just the first element in a list eg: payments[0].get('key')
                    if re.match(self.re_check_list_specific_element, single_location):
                        location_name = re.search(self.re_list_location_name, single_location).group()
                        location_index = int(re.search(self.re_list_location_index, single_location).group())
                        try:
                            current_object = current_object.get(location_name)[location_index]
                        except IndexError:
                            print(f'Warning: indexerror {location_name}[{location_index}] does not exist in {location}. First column is {first_column}, data is {output_row.get(first_column)}')
                            if mapperlocation.if_missing == 'fail':
                                raise RequiredElementMissingError(f'Required element {location_name}[{location_index}] does not exist in {location}. First column is {first_column}, data is {output_row.get(first_column)}')
                            if mapperlocation.if_missing == 'drop':
                                print('dropping row')
                                return None
                            else:
                                output_row.update({column: mapperlocation.coalesce_value})
                                break
                        
                    else:
                        if not hasattr(current_object, 'get') and type(current_object) == str and mapperlocation.attempt_json_serializing:
                            current_object = json.loads(current_object)
                            
                        current_object = current_object.get(single_location)
                        if current_object is None:
                            print(f'Warning: elementnotfound {single_location} does not exist in {location}. First column is {first_column}, data is {output_row.get(first_column)}')
                            if mapperlocation.if_missing == 'fail':
                                raise RequiredElementMissingError(f'Required element {single_location} does not exist in {location}. First column is {first_column}, data is {output_row.get(first_column)}')
                            elif mapperlocation.if_missing == 'drop':
                                print(f'dropping row')
                                return None
                            else:
                                output_row.update({column: mapperlocation.coalesce_value})
                                break
                        
                    output_row.update({column: current_object})
                
                if convert_function is not None:
                    output_row[column] = convert_function(output_row[column])

            except AttributeError:
                print(f'Warning: attributeerror {single_location} does not exist in {location}. First column is {first_column}, data is {output_row.get(first_column)}')
                if mapperlocation.if_missing == 'fail':
                    raise RequiredElementMissingError(f'Required element {single_location} does not exist in {location}. First column is {first_column}, data is {output_row.get(first_column)}')
                
                elif mapperlocation.if_missing == 'drop':
                    print('dropping row')
                    return None

                else:
                    output_row.update({column: mapperlocation.coalesce_value})
                    continue

        if not extra_data_in_front:
            output_row.update(extra_data)
        
        return output_row

    def _dump_to_records(self, in_rows, mapper_set, extra_data=None):
        if extra_data == None:
            extra_data = {}

        out_rows = []
        
        nested_list_mapper_set = []
        nested_list_base_location = set()

        single_mapper_set = []

        for mapper in mapper_set:
            if re.search(self.re_check_list_unlimited_element, mapper.location):
                nested_list_mapper_set.append(mapper)
                nested_list_base_location.add(re.match(self.re_check_list_base_location, mapper.location).group())
                nested_list_base_location_delimiter = mapper.delimiter
            else:
                single_mapper_set.append(mapper)

        if len(nested_list_base_location) > 1:
            raise AttributeError(f"This library does not currently supported multiple listed mappers in a single instance. Sorry. We found {nested_list_base_location}")

        if len(nested_list_base_location) > 0:
            nested_list_base_location = next(iter(nested_list_base_location))
            nested_list_base_location_elements = nested_list_base_location.split(nested_list_base_location_delimiter)
            nested_list_base_location = nested_list_base_location + f'[]{nested_list_base_location_delimiter}'

        for row in in_rows:
            output_row = self._dump_single_row(in_row=row, mapper_set=single_mapper_set, extra_data=extra_data)

            if len(nested_list_base_location) > 0:
                output_nested_rows = []
                nested_rows = row.copy()
                for element in nested_list_base_location_elements:
                    if element == 'root':
                        continue
                    else:
                        nested_rows = nested_rows.get(element, {})

                for nested_row in nested_rows:
                    output_nested_rows.append(self._dump_single_row(in_row=nested_row, mapper_set=nested_list_mapper_set, extra_data=output_row, extra_data_in_front=True, ignore_location_str=nested_list_base_location))

                out_rows.extend(output_nested_rows)
                
                if len(nested_rows) == 0 and output_row is not None:
                    out_rows.append(output_row)

            else:
                if output_row is not None:
                    out_rows.append(output_row)


        return out_rows

In [143]:
lower = lambda x: x.lower()

mapper = {
    'realtime_trips': [
        MapperLocation(location='root::id', column_name='message_id'),
        MapperLocation(location='root::tripUpdate::trip::tripId', column_name='trip_id', if_missing='drop'),
        MapperLocation(location='root::tripUpdate::trip::startTime', column_name='start_time'),
        MapperLocation(location='root::tripUpdate::trip::startDate', column_name='start_date'),
        MapperLocation(location='root::tripUpdate::trip::routeId', column_name='route_id'),
        MapperLocation(location='root::tripUpdate::trip::[transit_realtime.nyct_trip_descriptor]::trainId', column_name='train_id'),
        MapperLocation(location='root::tripUpdate::trip::[transit_realtime.nyct_trip_descriptor]::isAssigned', column_name='is_assigned'),
        MapperLocation(location='root::tripUpdate::trip::[transit_realtime.nyct_trip_descriptor]::direction', column_name='direction', coalesce_value='FUCK', convert_function=lower),
        MapperLocation(location='root::tripUpdate::stopTimeUpdate[]::arrival::time', column_name='scheduled_train_arrival_time'),
        MapperLocation(location='root::tripUpdate::stopTimeUpdate[]::departure::time', column_name='scheduled_train_departure_time'),
        MapperLocation(location='root::tripUpdate::stopTimeUpdate[]::stopId', column_name='scheduled_train_stop_id'),
        MapperLocation(location='root::tripUpdate::stopTimeUpdate[]::[transit_realtime.nyct_stop_time_update]::scheduledTrack', column_name='scheduled_train_scheduled_track'),
        MapperLocation(location='root::tripUpdate::stopTimeUpdate[]::[transit_realtime.nyct_stop_time_update]::actualTrack', column_name='scheduled_train_actual_track'),
    ],
    'vehicle_locations': [
        MapperLocation(location='root::id', column_name='message_id'),
        MapperLocation(location='root::vehicle::trip::tripId', column_name='trip_id'),
        MapperLocation(location='root::vehicle::trip::startTime', column_name='start_time'),
        MapperLocation(location='root::vehicle::trip::startDate', column_name='start_date'),
        MapperLocation(location='root::vehicle::trip::routeId', column_name='route_id'),
        MapperLocation(location='root::vehicle::trip::[transit_realtime.nyct_trip_descriptor]::trainId', column_name='train_id'),
        MapperLocation(location='root::vehicle::trip::[transit_realtime.nyct_trip_descriptor]::isAssigned', column_name='is_assigned'),
        MapperLocation(location='root::vehicle::trip::[transit_realtime.nyct_trip_descriptor]::direction', column_name='direction'),
        MapperLocation(location='root::vehicle::currentStopSequence', column_name='current_stop_sequence'),
        MapperLocation(location='root::vehicle::currentStatus', column_name='current_status'),
        MapperLocation(location='root::vehicle::timestamp', column_name='timestamp'),
        MapperLocation(location='root::vehicle::stopId', column_name='stop_id'),
    ]
}

with open('in.json', 'r') as f:
    in_json = json.loads(f.read())

realtime_trips_root = in_json.get('messages')
print(f'Got {len(realtime_trips_root)} realtime trips')

rt = DictMapper(in_dict_list=realtime_trips_root, mapper=mapper)

# realtime_trips_data = rt.get_records(table_name='realtime_trips')
# print(realtime_trips_data)

# vehicle_locations_data = rt.get_records(table_name='vehicle_locations')

autoflat = rt.auto_flatten_records()


Got 74 realtime trips


In [144]:
df = pd.DataFrame.from_records(autoflat)

<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'None

In [128]:
df

,id,tripUpdate_trip_tripId,tripUpdate_trip_startTime,tripUpdate_trip_startDate,tripUpdate_trip_routeId,tripUpdate_trip_[transit_realtime.nyct_trip_descriptor]_trainId,tripUpdate_trip_[transit_realtime.nyct_trip_descriptor]_isAssigned,tripUpdate_trip_[transit_realtime.nyct_trip_descriptor]_direction,tripUpdate_stopTimeUpdate,vehicle_trip_tripId,vehicle_trip_startTime,vehicle_trip_startDate,vehicle_trip_routeId,vehicle_trip_[transit_realtime.nyct_trip_descriptor]_trainId,vehicle_trip_[transit_realtime.nyct_trip_descriptor]_isAssigned,vehicle_trip_[transit_realtime.nyct_trip_descriptor]_direction,vehicle_currentStopSequence,vehicle_currentStatus,vehicle_timestamp,vehicle_stopId
0,000001N,013738_N..S,02:17:23,20231207,N,1N 0210+ DIT/STL,True,SOUTH,"[{'arrival': {'time': '1701938910'}, 'departur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000002N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,013738_N..S,02:17:23,20231207,N,1N 0210+ DIT/STL,True,SOUTH,38.0,STOPPED_AT,1701938910,N09S
2,000003N,013300_N..N,02:13:00,20231207,N,1N 0213 STL/DIT,True,NaN,"[{'arrival': {'time': '1701938900'}, 'departur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000004N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,013300_N..N,02:13:00,20231207,N,1N 0213 STL/DIT,True,NaN,44.0,STOPPED_AT,1701938910,R04N
4,000005N,016050_N..S,02:40:30,20231207,N,1N 0230+ DIT/STL,True,SOUTH,"[{'arrival': {'time': '1701938910'}, 'departur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,000008R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,026600_R..N,04:26:00,20231207,R,1R 0426 95S/WHL,NaN,NaN,NaN,NaN,1701941160,R45N
70,000009R,021600_R..N,03:36:00,20231207,R,1R 0336 95S/WHL,True,NaN,"[{'arrival': {'time': '1701938889'}, 'departur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,000010R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,021600_R..N,03:36:00,20231207,R,1R 0336 95S/WHL,True,NaN,6.0,STOPPED_AT,1701938889,R39N
72,000011R,028600_R..N,04:46:00,20231207,R,1R 0446 95S/WHL,NaN,NaN,"[{'arrival': {'time': '1701942360'}, 'departur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
import pandas as pd

df = pd.DataFrame.from_records(realtime_trips_data)

display(df)

,message_id,trip_id,start_time,start_date,route_id,train_id,is_assigned,direction,scheduled_train_arrival_time,scheduled_train_departure_time,scheduled_train_stop_id,scheduled_train_scheduled_track,scheduled_train_actual_track
0,000001N,013738_N..S,02:17:23,20231207,N,1N 0210+ DIT/STL,True,south,1701938910,1701938910,N09S,E1,E1
1,000001N,013738_N..S,02:17:23,20231207,N,1N 0210+ DIT/STL,True,south,1701939165,1701939165,N10S,E1,E1
2,000001N,013738_N..S,02:17:23,20231207,N,1N 0210+ DIT/STL,True,south,1701939645,1701939645,D43S,0B,0B
3,000003N,013300_N..N,02:13:00,20231207,N,1N 0213 STL/DIT,True,fuck,1701938900,1701938900,R05N,G2,G2
4,000003N,013300_N..N,02:13:00,20231207,N,1N 0213 STL/DIT,True,fuck,1701938910,1701938910,R04N,G2,G2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,000011R,028600_R..N,04:46:00,20231207,R,1R 0446 95S/WHL,None,fuck,1701943800,1701943800,R30N,B2,B2
869,000011R,028600_R..N,04:46:00,20231207,R,1R 0446 95S/WHL,None,fuck,1701943890,1701943890,R29N,B2,B2
870,000011R,028600_R..N,04:46:00,20231207,R,1R 0446 95S/WHL,None,fuck,1701943980,1701943980,R28N,B2,B2
871,000011R,028600_R..N,04:46:00,20231207,R,1R 0446 95S/WHL,None,fuck,1701944130,1701944130,R65N,B2,B2


In [46]:
df2 = pd.DataFrame.from_records(vehicle_locations_data)

df2

,message_id,trip_id,start_time,start_date,route_id,train_id,is_assigned,direction,current_stop_sequence,current_status,timestamp,stop_id
0,000001N,None,None,None,None,None,None,None,NaN,None,None,None
1,000002N,013738_N..S,02:17:23,20231207,N,1N 0210+ DIT/STL,True,SOUTH,38.0,STOPPED_AT,1701938910,N09S
2,000003N,None,None,None,None,None,None,None,NaN,None,None,None
3,000004N,013300_N..N,02:13:00,20231207,N,1N 0213 STL/DIT,True,None,44.0,STOPPED_AT,1701938910,R04N
4,000005N,None,None,None,None,None,None,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
69,000008R,026600_R..N,04:26:00,20231207,R,1R 0426 95S/WHL,None,None,NaN,None,1701941160,R45N
70,000009R,None,None,None,None,None,None,None,NaN,None,None,None
71,000010R,021600_R..N,03:36:00,20231207,R,1R 0336 95S/WHL,True,None,6.0,STOPPED_AT,1701938889,R39N
72,000011R,None,None,None,None,None,None,None,NaN,None,None,None


In [44]:
# df.to_csv('blah.csv', index=False)
df2.to_csv('blah2.csv', index=False)